In [84]:
import numpy as np
import time
from collections import defaultdict

In [85]:
living_reward = 0
hole_reward = 0
goal_reward = 1
p = 0.8
move_list = [0,1,2,3]
# 0 - up , 1 - down , 2 - left , 3 - right

In [108]:
with open('large_map.csv', 'r') as f:
    data = f.readlines()
for i in range(len(data)):
    data[i] = data[i].strip().split(',')
    # data[i] = [int(x) for x in data[i]]

In [109]:
state_list = [(i, j) for i in range(len(data)) for j in range(len(data[0]))]
map_left_x = state_list[0][0]
map_right_x = state_list[-1][0]
map_top_y = state_list[0][1]
map_bottom_y = state_list[-1][1]

In [110]:
def check_legal(current_pos, move):
    if move == 0:
        new_pos = (current_pos[0], current_pos[1]-1)
    elif move == 1:
        new_pos = (current_pos[0], current_pos[1]+1)
    elif move == 2:
        new_pos = (current_pos[0]-1, current_pos[1])
    elif move == 3:
        new_pos = (current_pos[0]+1, current_pos[1])
    if new_pos[0] < map_left_x or new_pos[0] > map_right_x:
        return False
    if new_pos[1] < map_top_y or new_pos[1] > map_bottom_y:
        return False
    return True

def update_pos(current_pos, move):
    if move == 0:
        new_pos = (current_pos[0], current_pos[1]-1)
    elif move == 1:
        new_pos = (current_pos[0], current_pos[1]+1)
    elif move == 2:
        new_pos = (current_pos[0]-1, current_pos[1])
    elif move == 3:
        new_pos = (current_pos[0]+1, current_pos[1])
    return new_pos

In [125]:
# defining the transition map
def construct_map(data, living_reward = 0, hole_reward = 0, goal_reward = 1, p = 0.8, move_list = [0,1,2,3]):
    transition_map = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda : 0)))
    reward_matrix = np.zeros((len(data), len(data[0])))
    for i in range(len(data)):
        for j in range(len(data[0])):
            if data[i][j] == 'H':
                reward_matrix[i][j] = hole_reward
            elif data[i][j] == 'G':
                reward_matrix[i][j] = goal_reward
            elif data[i][j] == 'S':
                reward_matrix[i][j] = living_reward
                start_state = (i, j)
            else:
                reward_matrix[i][j] = living_reward
            # the transition map is a S x A x S map
            # S is the number of states
            # A is the number of actions
            # p probability that desired action occurs otherwise p/3 probability that any of the other actions occur
            # if the action is not possible, then the state remains the same
            legal_list = np.zeros(len(move_list))
            for ind,move in enumerate(move_list):
                if check_legal((i,j), move):
                    legal_list[ind] = 1
            for ind,move in enumerate(move_list):
                if legal_list[ind] == 1:
                    transition_map[(i,j)][move][update_pos((i,j), move)] = p
                    for ind2,move2 in (enumerate(move_list)):
                        if ind2 != ind:
                            if legal_list[ind2] == 1:
                                transition_map[(i,j)][move][update_pos((i,j), move2)] += (1-p)/(len(move_list)-1)
                            else:
                                transition_map[(i,j)][move][(i,j)] += (1-p)/(len(move_list)-1)

                else :
                    transition_map[(i,j)][move][(i,j)] = p
                    for ind2,move2 in (enumerate(move_list)):
                        if ind2 != ind:
                            if legal_list[ind2] == 1:
                                transition_map[(i,j)][move][update_pos((i,j), move2)] += (1-p)/(len(move_list)-1)
                            else:
                                transition_map[(i,j)][move][(i,j)] += (1-p)/(len(move_list)-1)
    return transition_map, reward_matrix, start_state

def convert_to_matrix(state_list, action_list, transition_map):
    matrix = np.zeros((len(state_list), len(action_list), len(state_list)))
    dict_index_map = {state:i for i,state in enumerate(state_list)}
    for i in range(len(state_list)):
        dict_index_map[state_list[i]] = i
        for j in range(len(action_list)):
            for k in transition_map[state_list[i]][action_list[j]].keys():
                matrix[i][j][dict_index_map[k]] = transition_map[state_list[i]][action_list[j]][k]
    return matrix, dict_index_map
                

In [126]:
transition_map, reward_matrix,start = construct_map(data, living_reward, hole_reward, goal_reward, p, move_list)

In [127]:
mat_map, dict_ind_map = convert_to_matrix(state_list, move_list, transition_map)

In [115]:
dict_ind_inverse = {v: k for k, v in dict_ind_map.items()}

In [136]:
mat_map[0][1][np.where(mat_map[0][1] != 0)[0]]

array([0.13333333, 0.8       , 0.06666667])

In [ ]:
R_expected = np.zeros(len(data)*len(data[0]))
for i in range(len(data)):
    for j in range(len(data[0])):
        for 
        R_expected[i*len(data[0])+j] = reward_matrix[i][j]